<h2>File paths and imports</h2>

In [ ]:
from ui_lib import *

# path of the body detection model
model_path = "..."

# video paths:
# input video directory (without any annotation)
input_video_directory = "..."
# output (final version - with human interaction)
output_video_directory = "..."
# directory containing all the manual modifications

mannual_annotations_directory = f"{input_video_directory}/manual_annotations"
output_video_directory_temp = f"{output_video_directory}/temp"
raw_text_output_directory = f"{output_video_directory_temp}/raw_output"

# Create the directories if they do not exist yet
os.makedirs(input_video_directory, exist_ok=True)
os.makedirs(output_video_directory, exist_ok=True)
os.makedirs(mannual_annotations_directory, exist_ok=True)
os.makedirs(output_video_directory_temp, exist_ok=True)
os.makedirs(raw_text_output_directory, exist_ok=True)

<h2>First step:</h2>

In [ ]:
max_cosine_distance = 0.5       # maximal distance to match an object (lower = more strict)
nn_budget = None                # maximal buffer size
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)

# YOLOv8s initialisation
YOLOv8s = YOLO(model_path)

# DeepSORT initialisation
DeepSort = DeepSortTracker(metric)

# Osnet initialisation
Osnet = torchreid.models.build_model(name='osnet_x1_0', num_classes=751, pretrained=True)
Osnet.eval()

for input_video in os.listdir(input_video_directory):
    if input_video.endswith(".mp4") or input_video.endswith(".MP4"):

        full_video_path = os.path.join(input_video_directory, input_video)
        video_name = os.path.splitext(input_video)[0]

        # creation of the manual_annotation textfile if it doesn't exist yet
        annotation_file_path = f"{mannual_annotations_directory}/{video_name}.txt"
        try:
            with open(annotation_file_path, 'x') as f:
                print(f"{video_name}.txt automatically created in {mannual_annotations_directory}.")
        except FileExistsError:
            print(f"{video_name}.txt already present in {mannual_annotations_directory}.")

        # production of the textual outputs
        perform_tracking(
            input_video_path = full_video_path, 
            output_text_file_path = f"{raw_text_output_directory}/{video_name}.txt", 
            detection_model = YOLOv8s, 
            tracker = DeepSort,
            confidence_threshold = 0.5, 
            model_feature_extraction = Osnet
        )
        print(f"Annotations ready for video: {full_video_path}")

        # production of the visual output
        draw_bbox_from_file(
            file_path = f"{raw_text_output_directory}/{video_name}.txt", 
            input_video_path = full_video_path, 
            output_video_path = f"{output_video_directory_temp}/{video_name}-(temp).mp4",
            annotation_type="bbox",
            draw_frame_count=True
        )
        print(f"Treatment done: {full_video_path}")

<h2>Third step:</h2>

In [ ]:
for input_video in os.listdir(input_video_directory):
    if input_video.endswith(".mp4") or input_video.endswith(".MP4"):
        full_video_path = os.path.join(input_video_directory, input_video)
        video_name = os.path.splitext(input_video)[0]
        annotation_file = f"{mannual_annotations_directory}/{video_name}.txt"

        raw_reader = raw_tracking_data_reader(f"{raw_text_output_directory}/{video_name}.txt")

        try:
            edit_reader = modification_reader(annotation_file)
        except:
            print(f"Error: the manual annotation file related to the video <{full_video_path}> is not found. It must be located at <{annotation_file}>.")
            continue
        
        metadata_file_path = f"{output_video_directory}/{video_name}-treated.txt"
        output_video_path = f"{output_video_directory}/{video_name}-treated.mp4"
        writer = data_writer(metadata_file_path)

        # computation of the new metadata file
        modified_data = edit_raw_output(raw_reader, edit_reader) 

        # production of the textual output
        writer.write(modified_data)

        # production of the visual output
        draw_bbox_from_file(
            file_path = metadata_file_path, 
            input_video_path = full_video_path, 
            output_video_path = output_video_path,
            annotation_type="triangle"
        )